<a href="https://colab.research.google.com/github/AnDDoanf/learn_NLP/blob/master/notebooks/Programming_Assignment_4_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Named Entity Recognition with MIT Restaurant Dataset

Your name: Nguyen Van A

Student ID: USTH001

**Due: 23:59 19/3/2023**

## Task Description

In this assignment, you will train a NER Model using Conditional Random Fields (CRF) on and report the accuracy of your model on the test dataset.

You will use the [MIT Restaurant Dataset](https://groups.csail.mit.edu/sls/downloads/restaurant/) dataset to do the task.

## How to submit

- Attach notebook file (.ipynb) and submit your work to Google Class Room 
- Name your file as YourName_StudentID_Assignment4.ibynb. E.g., Nguyen_Van_A_ST099834_Assignment4.ipynb
- Write your name and student ID into this notebook
- Copying others' assignments is strictly prohibited.


## Install python-crfsuite

In [1]:
!pip install -q python-crfsuite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.0 MB/s eta 0:00:00


## Imports

In [2]:
from itertools import chain
import pycrfsuite

## Dataset

We will use [MIT Restaurant Dataset](https://groups.csail.mit.edu/sls/downloads/restaurant/) dataset.

The data set is already in CoNLL format. We will use the [train](https://groups.csail.mit.edu/sls/downloads/restaurant/restauranttrain.bio) data to create the NER model and evaluate the model on the [test](https://groups.csail.mit.edu/sls/downloads/restaurant/restauranttest.bio) data.

### Download data

In [3]:
%%capture
!rm -f restauranttrain.bio
!rm -f restauranttest.bio

!wget https://groups.csail.mit.edu/sls/downloads/restaurant/restauranttest.bio
!wget https://groups.csail.mit.edu/sls/downloads/restaurant/restauranttrain.bio

## Loading data (30 points)

In this part, you will load a data file into a list of sentences. Each sentence is a list of (word, tag) tuples.

**Note: Blank lines are used to seperate sentences.**

For instance, the sentence below will be loaded into a list

```
O	a
B-Rating	four
I-Rating	star
O	restaurant
B-Location	with
I-Location	a
B-Amenity	bar
```

You will complete the function below

In [84]:
## Add necessary import here

def load_data(file_path):
    """Load data into a list of list of (word, tag) tuples

    Args:
        file_path (str): Path to data

    Returns:
        sentences: list of (word, tag) tuples
    """
    sentences = []
    
    #TODO: Write your code here
    words = []
    tags = []
    for line in open(file_path, "r"):
      sentence = line.rstrip("\n").split("\t")
      if sentence[0] == '':
        sentences.append([tuple(words), tuple(tags)])
        words = []
        tags = []
        continue
      words.append(sentence[1])
      tags.append(sentence[0])
    return sentences

In [85]:
train_sents = load_data('restauranttrain.bio')
test_sents = load_data('restauranttest.bio')

Let's check the number of sentences in train and test data

In [86]:
len(train_sents)

7660

In [87]:
len(test_sents)

1521

In [88]:
train_sents[2]

[('5', 'star', 'resturants', 'in', 'my', 'town'),
 ('B-Rating', 'I-Rating', 'O', 'B-Location', 'I-Location', 'I-Location')]

## Features (50 points)

We can extract as many features as you want. You will implement following basic features.

※ Of course, you can add more features.

*Word identity (lowercase)*

- Previous word identity
- Current word identity
- Next word
- Previous word and current word combination. Concat the previous word the current word by '||'
- Current word and next word combination. Concat two words by '||'

*Word shapes*

- Word prefix and suffix (4 characters)
- The first character of the current word is the capital letter

**All you need to do is to complete the function `word2feature`.**

In [89]:
def word2features(sentence, i):
    """
    Arguments:
        sentence (list): list of words [w1, w2,...,w_n]
        i (int): index of the word
    Return:
        features (dict): dictionary of features
    """
    word = sentence[i].lower()
    prev_word = '' if i==0 else sentence[i-1].lower()
    next_word = '' if i==len(sentence)-1 else sentence[i+1].lower()
    features = {
        #TODO: Write your features here
        'word': word,
        'prev_word': prev_word,
        'next_word': next_word,
        'prev_cur_word': prev_word+"||"+word,
        'cur_next_word': word+"||"+next_word,
        'prefix': word[:4],
        'suffix': word[-4:],
        'cap_letter': word[0]
    }
    
    return features


def sent2features(sentence):
    """
    sentence is a list of words [w1, w2,...,w_n]
    """
    return [word2features(sentence, i) for i in range(len(sentence))]


def sent2labels(sentence):
    """
    sentence is a list of tuples (word, postag)
    """    
    return [tag for token, tag in sentence]

def untag(sentence):
    """
    sentence is a list of tuples (word, postag)
    """
    return [token for token,_ in sentence]

Let's try to extract features for the first sentence

In [90]:
train_sents[0]

[('2', 'start', 'restaurants', 'with', 'inside', 'dining'),
 ('B-Rating', 'I-Rating', 'O', 'O', 'B-Amenity', 'I-Amenity')]

In [91]:
sent2features(untag(train_sents)[0])[2]

{'word': 'restaurants',
 'prev_word': 'start',
 'next_word': 'with',
 'prev_cur_word': 'start||restaurants',
 'cur_next_word': 'restaurants||with',
 'prefix': 'rest',
 'suffix': 'ants',
 'cap_letter': 'r'}

### Create train/test data

In [92]:
X_train = [sent2features(untag(train_sents)[s]) for s in range(len(train_sents))]
y_train = sent2labels(train_sents)

X_test = [sent2features(untag(test_sents)[s]) for s in range(len(test_sents))]
y_test = sent2labels(test_sents)

## Training

In [93]:
%%time
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

CPU times: user 707 ms, sys: 5.11 ms, total: 712 ms
Wall time: 727 ms


In [94]:
#@title Set model parameters

max_iterations = 50 #@param[50, 20, 100]

trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': max_iterations,

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [95]:
%%time
trainer.train('mitrestaurant.crfsuite')

CPU times: user 6.61 s, sys: 12.2 ms, total: 6.62 s
Wall time: 6.64 s


## Evaluation (20 points)

We will use [seqeval](https://github.com/chakki-works/seqeval) package for evaluation NER result.

In [96]:
!pip install -q seqeval[cpu]

### Make Predictions

In [75]:
tagger = pycrfsuite.Tagger()
tagger.open('mitrestaurant.crfsuite')

In [97]:
example_sent = test_sents[0]
example_sent

[('a', 'four', 'star', 'restaurant', 'with', 'a', 'bar'),
 ('O', 'B-Rating', 'I-Rating', 'O', 'B-Location', 'I-Location', 'B-Amenity')]

In [98]:
print("Predicted:", ' '.join(tagger.tag(sent2features( example_sent[0]))))
print("Correct:  ", ' '.join(example_sent[1]))

Predicted: O B-Rating I-Rating O O O B-Amenity
Correct:   O B-Rating I-Rating O B-Location I-Location B-Amenity


In [99]:
%%time
y_pred = [tagger.tag(xseq) for xseq in X_test]

CPU times: user 135 ms, sys: 1.71 ms, total: 137 ms
Wall time: 145 ms


In [100]:
def convert(tuples):
    return [*tuples, ]
y_test_list = []
for i in range(len(y_test)):
  new_y_test = convert(y_test[i])
  y_test_list.append(new_y_test)

In [101]:
from seqeval.metrics import classification_report

print(classification_report(y_test_list, y_pred))

                 precision    recall  f1-score   support

        Amenity       0.70      0.65      0.67       533
        Cuisine       0.84      0.81      0.82       532
           Dish       0.76      0.72      0.74       288
          Hours       0.71      0.65      0.68       212
       Location       0.83      0.80      0.82       812
          Price       0.81      0.81      0.81       171
         Rating       0.78      0.77      0.77       201
Restaurant_Name       0.77      0.73      0.75       402

      micro avg       0.78      0.75      0.77      3151
      macro avg       0.77      0.74      0.76      3151
   weighted avg       0.78      0.75      0.77      3151



# References

1. Datasets for Entity Recognition: https://github.com/juand-r/entity-recognition-datasets
2. [sklearn-crfsuite tutorial](https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html#let-s-use-conll-2002-data-to-build-a-ner-system). 
3. [Quick Recipe: Build a POS tagger using a Conditional Random Field](https://nlpforhackers.io/crf-pos-tagger/)
4. [NLP Guide: Identifying Part of Speech Tags using Conditional Random Fields](https://medium.com/analytics-vidhya/pos-tagging-using-conditional-random-fields-92077e5eaa31)
5. [CRFsuite - Tutorial on Chunking Task](http://www.chokkan.org/software/crfsuite/tutorial.html)